# Word frequencies


Now that we have the abstracts in two nice neat .csv files, we need to download/import the packages needed, import the .csv files, and then can get on with the first part of the analysis. 

## Get ready 

As always, we start with a couple of code cells that load up and nickname some useful packages, then check file locations, then import files and check them. 


In [1]:
%%capture

# installing necessary pdf conversion packages via pip
# the '%%capture' at the top of this cell suppresses the output (which is normally quite long and annoying looking). 
# You can remove or comment it out if you prefer to see the output. 
!pip install nltk


In [2]:
%%capture

import os                         # os is a module for navigating your machine (e.g., file directories).
import nltk                       # nltk stands for natural language tool kit and is useful for text-mining. 
from nltk import word_tokenize    # and some of its key functions

nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.corpus import wordnet                    # Finally, things we need for lemmatising!
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
nltk.download('averaged_perceptron_tagger')        # Like a POS-tagger...
nltk.download('wordnet')
nltk.download('webtext')
from nltk.corpus import webtext

import pandas as pd
pd.set_option('display.max_colwidth', 200)
import numpy as np
import statistics
import datetime
date = datetime.date.today()

import codecs
import csv                        # csv is for importing and working with csv files

from collections import Counter

import statistics
import re                         # things we need for RegEx corrections
import matplotlib.pyplot as plt
import string 

import math 

English_punctuation = "-!\"#$%&()'*-–+,./:;<=>?@[\]^_`{|}~''“”"      # Things for removing punctuation, stopwords and empty strings
table_punctuation = str.maketrans('','', English_punctuation)

In [3]:
print(os.listdir("..\\output")  )                                # check 'results' folder is not empty/has correct stuff

['.ipynb_checkpoints', 'batch_2001.csv', 'batch_2002.csv', 'batch_2003.csv', 'batch_2004.csv', 'batch_2005.csv', 'batch_2006.csv', 'batch_2007.csv', 'batch_2008.csv', 'batch_2009.csv', 'batch_2010.csv', 'batch_2011.csv', 'batch_2012.csv', 'batch_2013.csv', 'batch_2014.csv', 'batch_2015.csv', 'batch_2016.csv', 'batch_2017.csv', 'batch_2018.csv', 'batch_2019.csv', 'batch_2020.csv', 'batch_2021.csv']


## Import

Having checked the contents of the output folder and seen the files we expected to see, we can now import and check them. 

In [4]:
batch_2001 = pd.read_csv('..\\output\\batch_2001.csv')            # start importing the files, each saved under its own name
batch_2002 = pd.read_csv('..\\output\\batch_2002.csv')             
batch_2003 = pd.read_csv('..\\output\\batch_2003.csv')             
batch_2004 = pd.read_csv('..\\output\\batch_2004.csv')            
batch_2005 = pd.read_csv('..\\output\\batch_2005.csv')             
batch_2006 = pd.read_csv('..\\output\\batch_2006.csv')             
batch_2007 = pd.read_csv('..\\output\\batch_2007.csv')            
batch_2008 = pd.read_csv('..\\output\\batch_2008.csv')             
batch_2009 = pd.read_csv('..\\output\\batch_2009.csv')            
batch_2010 = pd.read_csv('..\\output\\batch_2010.csv')            
batch_2011 = pd.read_csv('..\\output\\batch_2011.csv')             
batch_2012 = pd.read_csv('..\\output\\batch_2012.csv')             
batch_2013 = pd.read_csv('..\\output\\batch_2013.csv')            
batch_2014 = pd.read_csv('..\\output\\batch_2014.csv')             
batch_2015 = pd.read_csv('..\\output\\batch_2015.csv')             
batch_2016 = pd.read_csv('..\\output\\batch_2016.csv')            
batch_2017 = pd.read_csv('..\\output\\batch_2017.csv')             
batch_2018 = pd.read_csv('..\\output\\batch_2018.csv')             
batch_2019 = pd.read_csv('..\\output\\batch_2019.csv')            
batch_2020 = pd.read_csv('..\\output\\batch_2020.csv')             
batch_2021 = pd.read_csv('..\\output\\batch_2021.csv')             


## Count word frequencies - 'bag of words'

Now that we have some basic descriptive stats about how many abstracts were imported properly with text in the 'Text' column, we can get on to the actual natural language processing steps. The most basic NLP option is to count the most frequent words found in the two sets of abstracts - meaning we need to find the most frequent words found in ALL of the abstracts and then compare that to the most frequnet words found in only those abstracts that contain a keyword of interest. 

To this end, we use the 'bag of words' method which whacks all of the words from all of the texts together, turns them into 'tokens' then processes to make them as unified as possible by removing uppercase letters, punctuation, digits, empty strings, stop words (e.g. 'the', 'and', 'for', etc. ) and word forms (e.g. pluralisations, verb endings, etc. ). 

Let's demo this with a simple example. If the text we want to 'bag of words' is "The cat named Cat was one of 5 cats." it would become a list of stemmed word-tokens like 
'''[[cat]
[name]
[cat]
[be]
[cat]]''' 
and the most common word would obviously be '''[cat]'''. 

Applying the 'bag of words' method to our texts is not so trivial, but should also be more enlightening. We would expect that the most common words from all of the texts would be similar to, but not identical to, the most common words from only the abstracts that contain a keyword of interest.

This bag of words approach ignores years, session codes, authors and everything else. Subsetting the texts by those things might be useful later. 

In [51]:
REPLACEMENTS_1 = [
    ('mutations', 'mutation' ),
    ('variants', "variant"),
    ('changes', "change"),
    ('alterations', "alteration"),
    ('diseases', "disease"), 
    ('disorders', "disorder"),
    ('illnesses', "illness"), 
    ('conditions', "condition"),
    ('diagnoses', "diagnosis"), 
    ('syndromes', "syndrome"),
    ('patients', "patient"),
    ('individuals', "individual"),
    ('people', "person"),
    ('probands', "proband"), 
    ('subjects', "subject"), 
    ('cases', "case"), 
    ('normals', "normal"), 
    ('typicals', "typical"), 
    ('wilds', "wild"), 
    ('types', "type"), 
    ('abnormals', "abnormal"), 
    ('atypicals', "atypical"), 
    ]

REPLACEMENTS_2 = [
    ('gene change', "genechange"), 
    ('gene alteration', "genealteration"), 
    ('suffering from', "sufferingfrom"), 
    ('living with', "livingwith"), 
    ('wild type', "wildtype"), 
    ]

In [7]:
print(sorted(stop_words))                                # OPTIONAL: check what counts as a stopword if you want to see

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some',

In [78]:
sets = [1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5]
words = ['mutation', 'variant', 'genechange', 'genealteration', 
         'disease', 'disorder', 'illness', 'condition', 'diagnosis', 'syndrome', 
         'patient', 'individual', 'person', 'proband', 'subject', 'case', 
         'affected', 'diagnosed', 'sufferingfrom', 'impacted', 'livingwith', 
         'normal', 'typical', 'wildtype', 'abnormal', 'atypical']

target_word_dataframe = pd.DataFrame(list(zip(sets, words)), columns=['set','words'])
target_word_dataframe

,set,words
0,1,mutation
1,1,variant
2,1,genechange
3,1,genealteration
4,2,disease
5,2,disorder
6,2,illness
7,2,condition
8,2,diagnosis
9,2,syndrome


In [122]:
def create_a_bag_of_words (input, target_words, year):
    holding_string = ""                                                        # that creates a temporary variable
    for text in input['Text']:                                                 # looks at the 'Text' column for the input
        holding_string += text                                                 # fills up the temp variable with the text
    for plural, singular in REPLACEMENTS_1:
        holding_string = holding_string.replace(plural, singular)
    for multi, single in REPLACEMENTS_2:
        holding_string = holding_string.replace(multi, single)
    holding_string = word_tokenize(holding_string)                             # word tokenises that text
    holding_string = [word.lower() for word in holding_string]                 # remove uppercase letters
    holding_string = [w.translate(table_punctuation) for w in holding_string]  # removes punctuation
    holding_string = (list(filter(lambda x: x, holding_string)))               # removes andy empty strings
    holding_string = [token for token in holding_string if not token.isdigit()]  # removes digits
    holding_string = [token for token in holding_string if token not in stop_words]  # removes stopwords
    list_for_count = []                                                              # and creates an empty list        
    for token in holding_string:                                         # then iterates over the tokens
        if token in target_words.values:
            list_for_count.append(token)                                     # appending them to the list
    counts = Counter(list_for_count)                                     # applies the Counter function imported earlier 
    year  = pd.DataFrame.from_records(list(dict(counts).items()), columns=['words',year])
    return year

In [123]:
bag_of_2001 = create_a_bag_of_words(batch_2001, target_word_dataframe, '2001')

bag_of_2001


,words,2001
0,mutation,3035
1,disease,1343
2,patient,2778
3,individual,658
4,illness,15
5,diagnosis,655
6,affected,504
7,variant,295
8,case,1666
9,typical,84


In [46]:
print(counts['autist'])

[('gene', 4207),
 ('mutat', 3295),
 ('patient', 2851),
 ('genet', 2270),
 ('chromosom', 2026)]

In [40]:
def bag_of_words_analysis_1(input, target_words_list):     # define a 'bag of words' function with 2 arguments, an input and a quantity 
    holding_string = ""                                                        # that creates a temporary variable
    for text in input['Text']:                                                 # looks at the 'Text' column for the input
        holding_string += text                                                 # fills up the temp variable with the text
    holding_string = word_tokenize(holding_string)                             # word tokenises that text
    holding_string = [word.lower() for word in holding_string]                 # remove uppercase letters
    holding_string = [w.translate(table_punctuation) for w in holding_string]  # removes punctuation
    holding_string = (list(filter(lambda x: x, holding_string)))               # removes andy empty strings
    holding_string = [token for token in holding_string if not token.isdigit()]  # removes digits
    holding_string = [token for token in holding_string if token not in stop_words]  # removes stopwords
    list_for_count = []                                                              # and creates an empty list
    for token in holding_string:                                         # then iterates over the tokens
        list_for_count.append(token)                                     # appending them to the list
    counts = Counter(list_for_count)                                     # applies the Counter function imported earlier 
    return counts.most_common(how_many)                                  # and returns the tokens with highest counts 
                                                                         # up to the quantity specified as an argument

In [44]:
def bag_of_words_analysis(input, how_many):     # define a 'bag of words' function with 2 arguments, an input and a quantity 
    holding_string = ""                                                        # that creates a temporary variable
    for text in input['Text']:                                                 # looks at the 'Text' column for the input
        holding_string += text                                                 # fills up the temp variable with the text
    holding_string = word_tokenize(holding_string)                             # word tokenises that text
    holding_string = [word.lower() for word in holding_string]                 # remove uppercase letters
    holding_string = [w.translate(table_punctuation) for w in holding_string]  # removes punctuation
    holding_string = (list(filter(lambda x: x, holding_string)))               # removes andy empty strings
    holding_string = [token for token in holding_string if not token.isdigit()]  # removes digits
    holding_string = [token for token in holding_string if token not in stop_words]  # removes stopwords
    holding_string = [porter.stem(token) for token in holding_string]                # stems the word-tokens
    list_for_count = []                                                              # and creates an empty list
    for token in holding_string:                                         # then iterates over the tokens
        list_for_count.append(token)                                     # appending them to the list
    counts = Counter(list_for_count)                                     # applies the Counter function imported earlier 
    return counts.most_common(how_many)                                  # and returns the tokens with highest counts 
                                                                         # up to the quantity specified as an argument

In [7]:
test = bag_of_words_analysis(batch_2001, 20)   # apply bag of words function to all texts, and save the output table
                                                           # this will take a while. 
    
test

[('gene', 4207),
 ('mutat', 3295),
 ('patient', 2851),
 ('genet', 2270),
 ('chromosom', 2026),
 ('studi', 1897),
 ('famili', 1743),
 ('case', 1683),
 ('use', 1643),
 ('analysi', 1576),
 ('result', 1448),
 ('diseas', 1362),
 ('region', 1303),
 ('two', 1274),
 ('one', 1271),
 ('cell', 1270),
 ('allel', 1201),
 ('syndrom', 1159),
 ('popul', 1153),
 ('differ', 1107)]

### Steps to take

* revise the bag_of_words_analysis function to group by year? ALternatively, create one bag of words for each year. 
* write a new function that scans the year-bags-of-words for all words on an input list
* save the output of that function to a list (or .csv?) 
* use the outputs to create graphs that track the popularity of all the words on that list over time



In [9]:
counts = Counter(stemmed)
print(type(counts))

NameError: name 'stemmed' is not defined

In [ ]:
print(counts.most_common(35))

In [8]:
print(counts['autist'])
print(counts['asd'])
print(counts['asperg'])
print(counts['autism'])

NameError: name 'counts' is not defined